In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import os
import shutil

root_src_dir = "/content/gdrive/MyDrive/CE_Project/model"
root_dst_dir = "/content/model"

for src_dir, dirs, files in os.walk(root_src_dir):
    dst_dir = src_dir.replace(root_src_dir, root_dst_dir, 1)
    if not os.path.exists(dst_dir):
        # print('making new dir.')
        os.makedirs(dst_dir)
    for file_ in files:
        src_file = os.path.join(src_dir, file_)
        dst_file = os.path.join(dst_dir, file_)
        if os.path.exists(dst_file):
            # print('file already exist.')
            try:
                # print('removing old file.')
                os.remove(dst_file)
            except PermissionError as exc:
                os.chmod(dst_file, stat.S_IWUSR)
                os.remove(dst_file)
        shutil.copy(src_file, dst_dir)

#Attempt 1 Fail output is Kmodel v5

In [ ]:
%cd /content/model/yolov5

/content/model/yolov5


In [ ]:
# https://github.com/ultralytics/yolov5/issues/5707
%pip install --upgrade flatbuffers==1.12 # downgrade from v2 to v1.12

  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 2.0
    Uninstalling flatbuffers-2.0:
      Successfully uninstalled flatbuffers-2.0


In [ ]:
!python export.py --weights /content/model/yolov5/runs/train/exp3/weights/best.pt --img 224 --include onnx tflite

export: data=data/coco128.yaml, weights=['/content/model/yolov5/runs/train/exp3/weights/best.pt'], imgsz=[224], batch_size=1, device=cpu, half=False, inplace=False, train=False, optimize=False, int8=False, dynamic=False, simplify=False, opset=14, verbose=False, workspace=4, topk_per_class=100, topk_all=100, iou_thres=0.45, conf_thres=0.25, include=['onnx', 'tflite']
YOLOv5 🚀 v6.0-127-g554f782 torch 1.10.0+cu111 CPU

Fusing layers... 
Model Summary: 213 layers, 7066762 parameters, 0 gradients

PyTorch: starting from /content/model/yolov5/runs/train/exp3/weights/best.pt (57.2 MB)

ONNX: starting export with onnx 1.10.2...
/content/model/yolov5/models/yolo.py:57: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if self.onnx_dynamic or self.grid[i].shape[2:4] != x[i].shape[2:

In [ ]:
!pip install onnx-simplifier
!pip install onnx
!pip install nncase

In [ ]:
import os
import onnxsim
import onnx
import nncase

def parse_model_input_output(model_file):
    onnx_model = onnx.load(model_file)
    input_all = [node.name for node in onnx_model.graph.input]
    input_initializer = [node.name for node in onnx_model.graph.initializer]
    input_names = list(set(input_all) - set(input_initializer))
    input_tensors = [node for node in onnx_model.graph.input if node.name in input_names]

    # input
    inputs= []
    for _, e in enumerate(input_tensors):
        onnx_type = e.type.tensor_type
        input_dict = {}
        input_dict['name'] = e.name
        input_dict['dtype'] = onnx.mapping.TENSOR_TYPE_TO_NP_TYPE[onnx_type.elem_type]
        input_dict['shape'] = [(i.dim_value if i.dim_value != 0 else d) for i, d in zip(
            onnx_type.shape.dim, [1, 3, 224, 224])]
        inputs.append(input_dict)


    return onnx_model, inputs

def onnx_simplify(model_file):
    onnx_model, inputs = parse_model_input_output(model_file)
    onnx_model = onnx.shape_inference.infer_shapes(onnx_model)
    input_shapes = {}
    for input in inputs:
        input_shapes[input['name']] = input['shape']

    onnx_model, check = onnxsim.simplify(onnx_model, input_shapes=input_shapes)
    assert check, "Simplified ONNX model could not be validated"

    model_file = os.path.join(os.path.dirname(model_file), 'simplified.onnx')
    onnx.save_model(onnx_model, model_file)
    return model_file


def read_model_file(model_file):
    with open(model_file, 'rb') as f:
        model_content = f.read()
    return model_content


def main():
    model_file = '/content/model/yolov5/runs/train/exp3/weights/best.onnx'
    target = 'k210'

    # onnx simplify
    model_file = onnx_simplify(model_file)

    # compile_options
    compile_options = nncase.CompileOptions()
    compile_options.target = target
    compile_options.dump_ir = True
    compile_options.dump_asm = True
    compile_options.dump_dir = 'tmp'

    # compiler
    compiler = nncase.Compiler(compile_options)

    # import_options
    import_options = nncase.ImportOptions()

    # import
    model_content = read_model_file(model_file)
    compiler.import_onnx(model_content, import_options)

    # compile
    compiler.compile()

    # kmodel
    kmodel = compiler.gencode_tobytes()
    with open('/content/model/yolov5/runs/train/exp3/weights/best_onnx.kmodel', 'wb') as f:
        f.write(kmodel)

# if __name__ == '__main__':
main()

In [ ]:
import nncase

def read_model_file(model_file):
    with open(model_file, 'rb') as f:
        model_content = f.read()
    return model_content

def main():
    model='/content/model/yolov5/runs/train/exp3/weights/best-fp32.tflite'
    target = 'k210'

    # compile_options
    compile_options = nncase.CompileOptions()
    compile_options.target = target
    compile_options.dump_ir = True
    compile_options.dump_asm = True
    compile_options.dump_dir = 'tmp'

    # compiler
    compiler = nncase.Compiler(compile_options)

    # import_options
    import_options = nncase.ImportOptions()

    # import
    model_content = read_model_file(model)
    compiler.import_tflite(model_content, import_options)

    # compile
    compiler.compile()

    # kmodel
    kmodel = compiler.gencode_tobytes()
    with open('/content/model/yolov5/runs/train/exp3/weights/best_tflite.kmodel', 'wb') as f:
        f.write(kmodel)

# if __name__ == '__main__':
main()

In [ ]:
import nncase

def read_model_file(model_file):
    with open(model_file, 'rb') as f:
        model_content = f.read()
    return model_content

def main():
    model='/content/model/yolov5/runs/train/exp3/weights/best-fp32.tflite'
    target = 'k210'

    # compile_options
    compile_options = nncase.CompileOptions()
    compile_options.target = target
    compile_options.input_type = 'float32'  # or 'uint8' 'int8' 'float32'
    compile_options.preprocess = True # if False, the args below will unworked
    compile_options.swapRB = True
    compile_options.input_shape = [1,224,224,3] # keep layout same as input layout
    compile_options.input_layout = 'NHWC'
    compile_options.output_layout = 'NHWC'
    compile_options.mean = [0,0,0]
    compile_options.std = [1,1,1]
    compile_options.input_range = [0,1]
    compile_options.letterbox_value = 114. # pad what you want
    compile_options.dump_ir = True
    compile_options.dump_asm = True
    compile_options.dump_dir = 'tmp'

    # compiler
    compiler = nncase.Compiler(compile_options)

    # import_options
    import_options = nncase.ImportOptions()

    # import
    model_content = read_model_file(model)
    compiler.import_tflite(model_content, import_options)

    # compile
    compiler.compile()

    # kmodel
    kmodel = compiler.gencode_tobytes()
    with open('/content/model/yolov5/runs/train/exp3/weights/best_tflite_preprocess.kmodel', 'wb') as f:
        f.write(kmodel)

if __name__ == '__main__':
    main()

In [ ]:
#save to drive
root_src_dir = "/content/model"
root_dst_dir = "/content/gdrive/MyDrive/CE_Project/model"

for src_dir, dirs, files in os.walk(root_src_dir):
    dst_dir = src_dir.replace(root_src_dir, root_dst_dir, 1)
    if not os.path.exists(dst_dir):
        # print('making new dir.')
        os.makedirs(dst_dir)
    for file_ in files:
        src_file = os.path.join(src_dir, file_)
        dst_file = os.path.join(dst_dir, file_)
        if os.path.exists(dst_file):
            # print('file already exist.')
            try:
                # print('removing old file.')
                os.remove(dst_file)
            except PermissionError as exc:
                os.chmod(dst_file, stat.S_IWUSR)
                os.remove(dst_file)
        shutil.copy(src_file, dst_dir)

#Attepmt 2 use NCC v0.2.0 beta4

In [3]:
%cd /content/model/ncc

/content/model/ncc


In [15]:
!sudo pwd

/content/model/ncc


In [24]:
!./ncc best.onnx best-onnx.kmodel -i onnx -o kmodel -t k210 --dataset images

/bin/bash: ./ncc: Permission denied


#Attempt 3 kendryte-standalone-sdk

https://github.com/kendryte/kendryte-standalone-sdk

In [26]:
%cd /content/model/SDK

/content/model/SDK


In [28]:
!git clone https://github.com/kendryte/kendryte-standalone-sdk

Cloning into 'kendryte-standalone-sdk'...
remote: Enumerating objects: 3937, done.
remote: Counting objects: 100% (337/337), done.
remote: Compressing objects: 100% (147/147), done.
remote: Total 3937 (delta 186), reused 297 (delta 161), pack-reused 3600
Receiving objects: 100% (3937/3937), 10.79 MiB | 18.95 MiB/s, done.
Resolving deltas: 100% (2713/2713), done.


In [30]:
%cd /content/model/SDK/kendryte-standalone-sdk

/content/model/SDK/kendryte-standalone-sdk


In [33]:
%cd src

/content/model/SDK/kendryte-standalone-sdk/src


In [ ]:
cd